In [ ]:
#Extra
#df = df.repartition(npartitions=49)
#df.head()
#df.npartitions
#df = df.shuffle(df.columns[4])
#df = df.get_partition(26)
#df = df.shuffle(df.columns[4])
# x = df.iloc[:,7].to_dask_array(lengths=True)
# y = df.iloc[:,4].to_dask_array(lengths=True)
# x = x.compute()
# y = y.compute()
# y = y.reshape(-1,1)
#tx = np.array(list(map(preprocess, x)))
# x = np.asarray(x).astype(np.float32)
# y = np.asarray(y).astype(np.float32)
# num_samples = x_dask.shape[0]
# indices = da.arange(num_samples, chunks=batch_size)
# for start in range(0, num_samples, batch_size):
#     end = start + batch_size
#     batch_indices = indices[start:end].compute()
#     yield x_dask[batch_indices], y_dask[batch_indices]

In [ ]:
!gcloud init

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import dask.dataframe as dd
from dask.array.slicing import shuffle_slice
import dask.array as da
import os

In [ ]:
df = dd.read_csv('train_data.csv', dtype={7: 'object', 4: 'object'}, sample=1000000, header=None)

In [ ]:
x = df.iloc[:,7].to_dask_array(lengths=True)
y = df.iloc[:,4].to_dask_array(lengths=True)

In [ ]:
x = x[1:]
y = y[1:]

In [ ]:
from sklearn.utils import shuffle
x, y = shuffle(x, y, random_state=0)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/__init__.py:185: PerformanceWarning: Slicing with an out-of-order index is generating 82 times more chunks
  return array[key] if axis == 0 else array[:, key]


In [ ]:
def preprocess_block(block):
    def preprocess(arr):
        t = np.array(arr.strip('][').split(' '))
        t = t[t != ''].astype(np.float32).reshape(256, 256, 3)
        return t
    b = da.array(list(map(preprocess, block))).compute()
    return b

In [ ]:
tx = x.map_blocks(preprocess_block, meta=da.array((), dtype=np.float32))

In [ ]:
# x = x.astype(np.float32)
y = y.astype(np.float32)

In [ ]:
def generate_batches(x_dask, y_dask, batch_size):
    for i in range(x_dask.numblocks[0]):
      yield x_dask.blocks[i], y_dask.blocks[i]

In [ ]:
model_base = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=[256, 256, 3],
    pooling=None,
    classes=2,
    classifier_activation='sigmoid'
)

In [ ]:
for layer in model_base.layers:
    layer.trainable = False

model = keras.models.Sequential()
model.add(model_base)
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))


opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics = ["accuracy", tf.metrics.AUC()])

In [ ]:
epochs = 5
data_batch_size = 82
cnt = 0

for epoch in range(epochs):
  print(epoch)
  for x_batch, y_batch in generate_batches(tx, y, data_batch_size):
    cnt+=1
    history = model.train_on_batch(
        x_batch.compute(),
        y_batch.compute()
        )
    if cnt%100==0:
      print(history)

In [ ]:
# f = open("training_output.txt", "w")
# print(cap, file=f)
# f.close()

# !gsutil cp training_output.txt gs://paper-2-data/

In [ ]:
model.save('inception_resnet_v2.model')

In [ ]:
df_test = dd.read_csv('test_data.csv', dtype={7: 'object', 4: 'object', 3:'object'}, sample=1000000, header=None)

In [ ]:
x_test = df_test.iloc[:,7].to_dask_array(lengths=True)
y_test = df_test.iloc[:,4].to_dask_array(lengths=True)

In [ ]:
x_test = x_test[1:]

In [ ]:
y_test = y_test[1:]

In [ ]:
tx_test = x_test.map_blocks(preprocess_block, meta=da.array((), dtype=np.float32))

In [ ]:
# x = x.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
xt1 = tx_test.blocks[0].compute()
yt1 = y_test.blocks[0].compute()

In [ ]:
xt2 = tx_test.blocks[244].compute()
yt2 = y_test.blocks[244].compute()

In [ ]:
%%capture cap

model.evaluate(xt1, yt1)

In [ ]:
f = open("test1_output.txt", "w")
print(cap, file=f)
f.close()

!gsutil cp test1_output.txt gs://paper-2-data/

In [ ]:
model.predict(xt1)

In [ ]:
%%capture cap

model.evaluate(xt2, yt2)

In [ ]:
f = open("test2_output.txt", "w")
print(cap, file=f)
f.close()

!gsutil cp test2_output.txt gs://paper-2-data/

In [ ]:
model.predict(xt2)